In [19]:
import pickle
import pandas as pd
from nltk.stem import WordNetLemmatizer
from textblob import Word

In [20]:
from newspaper import Article

url = 'https://www.foxnews.com/politics/supreme-court-strikes-down-state-ban-on-taxpayer-funding-for-religious-schools'
article = Article(url)
article.download()
article.parse()

In [21]:
articles = {'title': [article.title],'text': [article.text]}
df = pd.DataFrame(articles)

In [22]:
df.head()

,title,text
0,Supreme Court strikes down state ban on taxpay...,The Supreme Court on Tuesday struck down a ban...


In [23]:
def clean_dataset(X):
    #remove digits 
    #remove words less than 3 characters 
    #remove punctuation

    X['clean_title'] = X['title'].str.replace('\d+', ' ') # for digits
    X['clean_title'] = X['clean_title'].str.replace(r'(\b\w{1,2}\b)', ' ') # for words less than 3 characters
    X['clean_title'] = X['clean_title'].str.replace('[^\w\s]', ' ') # for punctuation 
    
    X['clean_text'] = X['text'].str.replace('\d+', ' ') # for digits
    X['clean_text'] = X['clean_text'].str.replace(r'(\b\w{1,2}\b)', ' ') # for words less than 3 characters
    X['clean_text'] = X['clean_text'].str.replace('[^\w\s]', ' ') # for punctuation 
    #lemmatization 
    X['clean_title'] = X['clean_title'] .apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
    X['clean_text'] = X['clean_text'] .apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
    return X

def makeCV_vector(df):
    #loading CountVec object
    cv_vector_title = pickle.load(open('cv_1-3_vector_title.sav','rb'))
    cv_vector_text = pickle.load(open('cv_1-3_vector_text.sav','rb'))
    
    vector_title = cv_vector_title.transform(df['clean_title'].values.astype('U'))
    vectorized_title = pd.DataFrame(vector_test_title.toarray(), columns=cv_vector_title.get_feature_names())
    
    vector_text = cv_vector_text.transform(df['clean_text'].values.astype('U'))
    vectorized_text = pd.DataFrame(vector_test_text.toarray(), columns=cv_vector_text.get_feature_names())
    
    #combine test title and text
    vectorized_cv = pd.concat([vectorized_title, vectorized_text], axis=1)
    return vectorized_cv
    
def makeTFIDF_vector(df):
    #loading TF-IDF object
    tfidf_vector_title = pickle.load(open('tfidf_1-3_vector_title.sav','rb'))
    tfidf_vector_text = pickle.load(open('tfidf_1-3_vector_text.sav','rb'))
    
    vector_title = tfidf_vector_title.transform(df['clean_title'].values.astype('U'))
    vectorized_title = pd.DataFrame(vector_title.toarray(), columns=tfidf_vector_title.get_feature_names())

    vector_text = tfidf_vector_text.transform(df['clean_text'].values.astype('U'))
    vectorized_text = pd.DataFrame(vector_text.toarray(), columns=tfidf_vector_text.get_feature_names())
    
    vectorized_tfidf = pd.concat([vectorized_title, vectorized_text], axis=1)
    return vectorized_tfidf

In [24]:
df = clean_dataset(df)
vectorized_cv = makeCV_vector(df)
vectorized_tfidf = makeTFIDF_vector(df)

In [25]:
#loading SVM model
svm = pickle.load(open('SVM_tfidf_1-3.sav','rb'))

#loading Random Forest model
rf = pickle.load(open('RandomForest_model_countvec_trigram.sav','rb'))

#loading other models.....

/Users/chi/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.22.1 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/chi/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.22.1 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [26]:
prediction = svm.predict_proba(vectorized_tfidf)

In [27]:
print("Trust rating: %.0f" %(prediction[0][1]*100))

Trust rating: 62


In [28]:
prediction2 = rf.predict_proba(vectorized_cv)

In [29]:
print("Trust rating: %.0f" %(prediction2[0][1]*100))

Trust rating: 47


In [ ]:
#other models' prediction.....